# 3pt data exploration

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from copy import deepcopy

In [3]:
sys.path.append(r"/Users/fordfishman/GitHub/3pt-shooting/code/python/")
# sys.path.append(r"/home/fordfishman/GitHub/3pt-shooting/code/python/")
from basketball_reference_scraper.teams import get_roster_stats



In [4]:
year_range = list( range(2010, 2022) )
team_abr = [
    'ATL',
    'BRK',
    'BOS',
    'CHA',
    'CHI',
    'CLE',
    'DAL',
    'DEN',
    'DET',
    'GSW',
    'HOU',
    'IND',
    'LAC',
    'LAL',
    'MEM',
    'MIA',
    'MIL',
    'MIN',
    'NOP',
    'NYK',
    'OKC',
    'ORL',
    'PHI',
    'PHO',
    'POR',
    'SAC',
    'SAS',
    'TOR',
    'UTA',
    'WAS',
    ]




In [5]:
def player_index(names, seasons):

    indices = list()

    for i in range(len(names)):

        name = names[i].replace(' ', '')
        indices.append( '%s%s' % (name, seasons[i]) )

    return indices
    
def get_data(mode):

    df = get_roster_stats('GSW', 2021, mode)
    df = pd.DataFrame().reindex(columns=df.columns)

    for year in year_range:
        for team in team_abr:

            df_i = get_roster_stats(team, year, mode)
            df_i['index'] = player_index(df_i.PLAYER, df_i.SEASON)
            df = df.append(df_i, ignore_index=True)

    return df


In [6]:
# df_adv = get_data('ADVANCED')
# df_tot = get_data('TOTALS')
# df_poss = get_data('PER_POSS')
# df_adv.to_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv", index=False)
# df_poss.to_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv", index=False)
# df_tot.to_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv", index=False)

In [7]:
df_poss = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv")
df_poss.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Unnamed: 29',
       'ORtg', 'DRtg', 'SEASON', 'index'],
      dtype='object')

have to get combine player data from in-season trades
then prune data for attempts
then prune for consecutive seasons

In [8]:
df_adv = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv")
df_adv.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'SEASON', 'index'],
      dtype='object')

In [9]:
df_tot = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv")
df_tot.columns

Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Unnamed: 29',
       'ORtg', 'DRtg', 'SEASON', 'eFG%', 'index'],
      dtype='object')

In [10]:
df_tot = df_tot.drop(['Unnamed: 29', 'DRtg', 'ORtg'], axis=1)
df_poss = df_poss.drop(['Unnamed: 29'], axis=1)
df_adv = df_adv.drop(['Unnamed: 19', 'Unnamed: 24'], axis=1)

remove duplicate columns, print out duplicates

In [11]:
for col in df_tot:
    
    if col in df_poss and df_tot[col].equals(df_poss[col]):

        df_poss = df_poss.drop([col], axis=1)
        print(col)
        
    if col in df_adv and df_tot[col].equals(df_adv[col]):
        
        df_adv = df_adv.drop([col], axis=1)
        print(col)


PLAYER
PLAYER
POS
POS
AGE
AGE
TEAM
TEAM
G
G
GS
MP
MP
FG%
3P%
2P%
FT%
SEASON
SEASON
index
index


rename possession columns

In [12]:
df_poss = df_poss.add_suffix('_poss')
# df_poss = df_poss.rename({"index_poss":"index"}, axis="columns")
# df_poss = df_poss.rename({"index":"key"}, axis="columns")
# df_adv = df_adv.rename({"index":"key"}, axis="columns")
# df_tot = df_tot.rename({"index":"key"}, axis="columns")


combine data frames

In [13]:
df_totposs = df_tot.join(df_poss, how='outer')
df_all = df_totposs.join(df_adv, how='outer')
df_all.columns


Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SEASON', 'eFG%',
       'index', 'FG_poss', 'FGA_poss', '3P_poss', '3PA_poss', '2P_poss',
       '2PA_poss', 'FT_poss', 'FTA_poss', 'ORB_poss', 'DRB_poss', 'TRB_poss',
       'AST_poss', 'STL_poss', 'BLK_poss', 'TOV_poss', 'PF_poss', 'PTS_poss',
       'ORtg_poss', 'DRtg_poss', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [14]:
df_trade = deepcopy(df_all)

# df_trade['trade'] = 0

idents = dict() # dictionary of id to list of indexes

for i, row in df_all.iterrows():

    ident = row['index']

    if not ident in idents:
        idents[ident] = [i]

    else:
        idents[ident].append(i)




need to fix so that each player year is a single row

In [15]:
num = df_trade.select_dtypes(include=['float64','int64']).columns
char = df_trade.select_dtypes(include=['object']).columns
totals = pd.Index(['G', 'GS', 'MP', 'FG', 'FGA', 
       '3P', '3PA', '2P', '2PA',  'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'])
aves_list = list()

for i in num:
    if i not in totals:
        aves_list.append(i)
aves = pd.Index(aves_list)

all_inds = list()

i = 0
for ident, inds in idents.items():

    inds = list(inds)

    if len(inds) > 1: # if a player was moved
        all_inds += inds
        
        char_i = df_trade.loc[inds, char].reset_index(drop=True)
        tot_i = df_trade.loc[inds, totals].reset_index(drop=True)
        ave_i = df_trade.loc[inds, aves].reset_index(drop=True)

        char_new = char_i.loc[0,:]
        tot_new = tot_i.sum(axis=0)

        games = np.sum(tot_new.G)
        games_r = tot_i.G/games # ratio of games played
        ave_new = np.matmul(games_r, ave_i)

        new_row = pd.concat([char_new, tot_new, ave_new], axis=0)
        # print(new_row)
        
        df_trade = df_trade.append(new_row, ignore_index = True)

df_trade = df_trade.drop(index=all_inds).reset_index(drop=True)



# print(m)

/var/folders/qb/f7nlg06s14v9pk0mfmmc714m0000gn/T/ipykernel_705/4138194036.py:32: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  ave_new = np.matmul(games_r, ave_i)


remove seasons below 100 3PTA

Add information about next year's shooting

In [17]:

seasons_sorted = sorted(list(set(df_trade.SEASON)))
player_dict = dict()

next_3pt = list()
next_imp = list()

for i in range(len(df_trade)):

    player = df_trade.loc[i, "PLAYER"]
    season = df_trade.loc[i, "SEASON"]
    pct = df_trade.loc[i, "3P%"]

    if player not in player_dict:
        player_dict[player] = {season:pct}
    
    else:
        player_dict[player][season] = pct

player_dict
# need to remove seasons below a certain number of threes taken also

{'Mike Bibby': {'2009-10': 0.389, '2011-12': 0.318, '2010-11': 0.433825},
 'Jason Collins': {'2009-10': 0.0,
  '2010-11': 1.0,
  '2011-12': nan,
  '2013-14': 0.0,
  '2012-13': nan},
 'Jamal Crawford': {'2009-10': 0.382,
  '2010-11': 0.341,
  '2011-12': 0.308,
  '2012-13': 0.376,
  '2013-14': 0.361,
  '2014-15': 0.327,
  '2015-16': 0.34,
  '2016-17': 0.36,
  '2017-18': 0.331,
  '2018-19': 0.332,
  '2019-20': 0.5},
 'Maurice Evans': {'2009-10': 0.337,
  '2011-12': 0.378,
  '2010-11': 0.32604109589041097},
 'Al Horford': {'2009-10': 1.0,
  '2010-11': 0.5,
  '2011-12': 0.0,
  '2012-13': 0.5,
  '2013-14': 0.364,
  '2014-15': 0.306,
  '2015-16': 0.344,
  '2016-17': 0.355,
  '2017-18': 0.429,
  '2018-19': 0.36,
  '2019-20': 0.35,
  '2020-21': 0.368},
 'Othello Hunter': {'2009-10': nan},
 'Joe Johnson': {'2009-10': 0.369,
  '2010-11': 0.297,
  '2011-12': 0.388,
  '2012-13': 0.375,
  '2013-14': 0.401,
  '2014-15': 0.359,
  '2016-17': 0.411,
  '2015-16': 0.3846296296296296,
  '2017-18': 0.276090

change season to be a numerical